In [1]:
!git clone https://github.com/lunadunkel/NivkhGloss.git

Cloning into 'NivkhGloss'...
remote: Enumerating objects: 665, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 665 (delta 0), reused 4 (delta 0), pack-reused 658 (from 2)
Receiving objects: 100% (665/665), 63.23 MiB | 15.23 MiB/s, done.
Resolving deltas: 100% (339/339), done.


In [2]:
from NivkhGloss.system.installation import *
from NivkhGloss.system.models import *

Установка navec...
navec установлен
Установка spacy...
spacy установлен


In [3]:
nlp = download_spacy_model()
navec = download_navec()

Установка модели ru_core_news_sm...
ru_core_news_sm установлен


Загрузка Navec модели: 100%|███████████▉| 51770/51771 [00:05<00:00, 9020.64KB/s]


In [4]:
with open('NivkhGloss/models/lemma_insert/trie.pkl', 'rb') as f:
    trie = pickle.load(f)

with open('NivkhGloss/vocabularies/glossing/stem_vocab.json') as file:
    stem_vocab = json.load(file)

In [12]:
segm_dir = 'bpe_attention_lstm_cnn'

segm_model = MorphSegmentationCNN(vocab_size=len(symb_vocab),
    labels_number=len(label_dict), hidden_dim=512, n_layers=3, dropout=0.4,
    device="cuda", window=(3, 6), bpe_vocab_size=2500, use_attention=True,
    use_lstm=True, use_bpe=True).to("cuda")

model_state_dict = torch.load('/content/NivkhGloss/models/pth/bpe_attention_lstm_cnn.pth')['model_state_dict']
segm_model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [13]:
postagging_dir = 'char_pos_tagging'

pos_model = PosTagger(word_embedding_dim=64, char_embedding_dim=32,
                         hidden_dim=128,  vocab_size=len(word_vocab),
                         char_vocab_size=len(char_vocab), labels_number=len(pos_label_vocab),
                         device="cuda", use_char_ids=True, dropout=0.2).to(device="cuda")


model_state_dict = torch.load('/content/NivkhGloss/models/pth/char_pos_tagging.pth')['model_state_dict']
pos_model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [14]:
glossing_dir = 'glossing'

gloss_model = BiLSTMTagger(len(morpheme_vocab), embed_dim=128,
                     dropout=0.5, bidirectional=False,
                     num_layers=2, hidden_dim=256,
                     output_dim=len(gloss_vocab), device="cuda").to("cuda")


model_state_dict = torch.load('/content/NivkhGloss/models/pth/glossing.pth')['model_state_dict']
gloss_model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [15]:
from NivkhGloss.system.GlossText import *

In [16]:
NivkhGlosser = GlossText(segm_model, pos_model, gloss_model,
                         symb_vocab, char_vocab, word_vocab,
                         pos_label_vocab, morpheme_vocab, gloss_vocab,
                         glosses_dictionary, stem_vocab,
                         trie, navec, nlp, device='cuda')

In [17]:
NivkhGlosser.gloss_sent('ӿы,	ӿарор̌	мулкхир̌	пыньх	лытр̌',
                        translation='Да. Затем в берестяной корзинке бульон рыбный варит')

{'segmentation': 'ӿы,\tӿа-рор̌\tмулкхир̌\tпыньх\tлыт-р̌',
 'glossing': 'UNK\tбыть.таким-CONV:ANT.3.SG\tUNK\tсуп\tделать-CONV.3.SG'}

In [ ]:
def clear_punctuation(line):
    line = '\t'.join(line.split())
    new_line = re.sub('\[[А-ЯЁ]+\:\]\t', '', line)
    new_line = re.sub('[\.\,\?\!\:\(\)]+', '', new_line)
    new_line = re.sub('^\t|\t$', '', new_line)
    new_line = new_line.lower()
    return new_line

In [ ]:
original = []
translation = []
with open('/content/n_r.txt', encoding='utf8') as file:
    for num, line in enumerate(file):
        line = '\t'.join(line.strip().split())
        if re.match('\d+\>\t', line):
            line = clear_punctuation(re.sub('\d+\>\t', '', line))
            original.append(line)
        elif re.match('\d+\=\t', line):
            line = clear_punctuation(re.sub('\d+\=\t', '', line))
            translation.append(line)

In [ ]:
glossed_text = NivkhGlosser.gloss_text(original, translation)
f = open('a200_tmn.txt', 'w', encoding='utf8')
f.writelines('\n\n'.join(glossed_text))
f.close()

# Замер качества

In [ ]:
def measure_segmentation(gold, predicted):
    """
    gold: правильная сегментация(разделённых табуляцией)
    predicted: предсказанная сегментация

    Возвращает метрики: precision, recall, f1, accuracy по частям, точность по словам
    """
    TP = FP = FN = 0
    equal_parts = total_parts = 0
    correct_words = 0
    total_numb_words = 0
    for gold_sent, pred_sent in zip(gold, predicted):
        gold_list = gold_sent.split('\t')
        predicted_list = pred_sent.split('\t')
        total_numb_words += len(gold_list)

        for gold, pred in zip(gold_list, predicted_list):
            gold_parts = gold.split('-')
            pred_parts = pred.split('-')

            # print(gold_parts)
            # print(pred_parts)
            # Сравниваем части
            common = [p for p in pred_parts if p in gold_parts]
            TP += len(common)
            FP += len(pred_parts) - len(common)
            FN += len(gold_parts) - len(common)

            # Точность по частям
            for g, p in zip(gold_parts, pred_parts):
                equal_parts += int(g == p)
            total_parts += len(gold_parts)

            # Полностью совпадающие слова
            correct_words += int(gold_parts == pred_parts)

    precision = TP / (TP + FP) if TP + FP > 0 else 0
    recall = TP / (TP + FN) if TP + FN > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0
    part_accuracy = equal_parts / total_parts if total_parts > 0 else 0
    word_accuracy = correct_words / total_numb_words if total_numb_words > 0 else 0



    metrics = [
        ("Точность", precision),
        ("Полнота", recall),
        ("F1-мера", f1),
        ("Корректность по частям", part_accuracy),
        ("Точность по словам", word_accuracy)
    ]

    return metrics

In [ ]:
gold_segmented = [x['segmented'] for x in final_test]
gold_glossing = [x['glossed'] for x in final_test]

In [ ]:
predictions = []
for x in final_test:
    predictions.append(NivkhGlosser.gloss_sent(x['original'], x['translation']))

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
len(predictions)

348

In [ ]:
len(gold_glossing)

348

In [ ]:
!pip install torchtext==0.6.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.18.0
    Uninstalling torchtext-0.18.0:
      Successfully uninstalled torchtext-0.18.0


In [ ]:
from torchtext.data.metrics import bleu_score

In [ ]:
from typing import List
from torchtext.data.metrics import bleu_score
import click
import json


def eval_accuracy(pred: List[List[str]], gold: List[List[str]]) -> dict:
    """Computes the average and overall accuracy, where predicted labels must be in the correct position in the list."""
    total_correct_predictions = 0
    total_tokens = 0
    summed_accuracies = 0

    for (entry_pred, entry_gold, i) in zip(pred, gold, range(len(gold))):
        entry_correct_predictions = 0

        for token_index in range(len(entry_gold)):
            # For each token, check if it matches
            if token_index < len(entry_pred) and entry_pred[token_index] == entry_gold[token_index] and entry_pred[token_index] != '[UNK]':
                entry_correct_predictions += 1

        entry_accuracy = (entry_correct_predictions / len(entry_gold))
        summed_accuracies += entry_accuracy

        total_correct_predictions += entry_correct_predictions
        total_tokens += len(entry_gold)

    total_entries = len(gold)
    average_accuracy = summed_accuracies / total_entries
    overall_accuracy = total_correct_predictions / total_tokens
    return {'average_accuracy': average_accuracy, 'accuracy': overall_accuracy}


def eval_stems_grams(pred: List[List[str]], gold: List[List[str]]) -> dict:
    perf = {'stem': {'correct': 0, 'pred': 0, 'gold': 0}, 'gram': {'correct': 0, 'pred': 0, 'gold': 0}}

    for (entry_pred, entry_gold) in zip(pred, gold):
        for token_index in range(len(entry_gold)):

            # We can determine if a token is a stem or gram by checking if it is all uppercase
            token_type = 'gram' if entry_gold[token_index].isupper() else 'stem'
            perf[token_type]['gold'] += 1

            if token_index < len(entry_pred):
                pred_token_type = 'gram' if entry_pred[token_index].isupper() else 'stem'
                perf[pred_token_type]['pred'] += 1

                if entry_pred[token_index] == entry_gold[token_index]:
                    # Correct prediction
                    perf[token_type]['correct'] += 1

    stem_perf = {'prec': 0 if perf['stem']['pred'] == 0 else perf['stem']['correct'] / perf['stem']['pred'],
                 'rec': perf['stem']['correct'] / perf['stem']['gold']}
    if (stem_perf['prec'] + stem_perf['rec']) == 0:
        stem_perf['f1'] = 0
    else:
        stem_perf['f1'] = 2 * (stem_perf['prec'] * stem_perf['rec']) / (stem_perf['prec'] + stem_perf['rec'])

    gram_perf = {'prec': 0 if perf['gram']['pred'] == 0 else perf['gram']['correct'] / perf['gram']['pred'],
                 'rec': perf['gram']['correct'] / perf['gram']['gold']}
    if (gram_perf['prec'] + gram_perf['rec']) == 0:
        gram_perf['f1'] = 0
    else:
        gram_perf['f1'] = 2 * (gram_perf['prec'] * gram_perf['rec']) / (gram_perf['prec'] + gram_perf['rec'])
    return {'stem': stem_perf, 'gram': gram_perf}


def eval_morpheme_glosses(pred_morphemes: List[List[str]], gold_morphemes: List[List[str]]):
    """Evaluates the performance at the morpheme level"""
    morpheme_eval = eval_accuracy(pred_morphemes, gold_morphemes)
    class_eval = eval_stems_grams(pred_morphemes, gold_morphemes)
    intersection = [len(set(x).intersection(set(y))) for x, y in zip(gold_morphemes, pred_morphemes)]
    print(sum(intersection) / len(intersection))
    bleu = bleu_score(pred_morphemes, [[line] for line in gold_morphemes])
    return {'morpheme_level': morpheme_eval, 'classes': class_eval, 'bleu': bleu}


def eval_word_glosses(pred_words: List[List[str]], gold_words: List[List[str]]):
    """Evaluates the performance at the morpheme level"""
    word_eval = eval_accuracy(pred_words, gold_words)
    bleu = bleu_score(pred_words, [[line] for line in gold_words])
    return {'word_level': word_eval, 'bleu': bleu}


def evaluate_igt(pred, gold):
    """Performs evaluation of a predicted IGT file"""

    pred_words = [line.split('\t') for line in pred]
    gold_words = [line.split('\t') for line in gold]
    word_eval = eval_accuracy(pred_words, gold_words)

    pred_morphemes = [re.split("\t|-", line) for line in pred]
    gold_morphemes = [re.split("\t|-", line) for line in gold]

    all_eval = {'word_level': word_eval, **eval_morpheme_glosses(pred_morphemes=pred_morphemes, gold_morphemes=gold_morphemes)}
    print(json.dumps(all_eval, sort_keys=True, indent=4))

In [ ]:
pred_segmented = [x['segmentation'] for x in predictions]
pred_glossing = [x['glossing'] for x in predictions]

In [ ]:
from collections import Counter

def compute_chrf_morpheme_as_ngram(true_segments, predicted_segments, beta=2):
    """
    Вычисляет метрику chrF++ для сегментированных данных, где морфемы рассматриваются как n-граммы.
    """
    def compute_fscore(precision, recall, beta):
        """ Вычисляет F-меру. """
        if precision + recall == 0:
            return 0.0
        return (1 + beta**2) * (precision * recall) / ((beta**2 * precision) + recall)

    ref_morphemes = Counter(true_segments)
    hyp_morphemes = Counter(predicted_segments)

    # совпадающие морфемы
    common_morphemes = ref_morphemes & hyp_morphemes
    matches = sum(common_morphemes.values())

    # общее количество морфем
    total_ref_morphemes = sum(ref_morphemes.values())
    total_hyp_morphemes = sum(hyp_morphemes.values())

    # точность и полнота
    precision = matches / total_hyp_morphemes if total_hyp_morphemes > 0 else 0
    recall = matches / total_ref_morphemes if total_ref_morphemes > 0 else 0

    # вычисление F-меры
    chrf_score = compute_fscore(precision, recall, beta)
    return chrf_score

In [ ]:
def compute_overall_chrf_morpheme_as_ngram(true_data, predicted_data, beta=2):
    """
    Вычисляет средний chrF++ для множества сегментированных данных.
    """
    scores = [
        compute_chrf_morpheme_as_ngram(true_segments, predicted_segments, beta)
        for true_segments, predicted_segments in zip(true_data, predicted_data)
    ]
    return sum(scores) / len(scores) if scores else 0.0

In [ ]:
all_gold_segmented = []
for x in gold_segmented:
    for word in x.split('\t'):
        all_gold_segmented.append(word.split('-'))

In [ ]:
all_pred_segmented = []
for x in pred_segmented:
    for word in x.split('\t'):
        all_pred_segmented.append(word.split('-'))

In [ ]:
overall_chrf = compute_overall_chrf_morpheme_as_ngram(all_gold_segmented, all_pred_segmented)

In [ ]:
overall_chrf

0.8876439587536151

In [ ]:
measure_segmentation(pred_segmented, gold_segmented)

[('Точность', 0.8709338929695698),
 ('Полнота', 0.8880171184022825),
 ('F1-мера', 0.8793925481193714),
 ('Корректность по частям', 0.8573466476462197),
 ('Точность по словам', 0.8619354838709677)]

In [ ]:
evaluate_igt(pred_glossing, gold_glossing)

3.5258620689655173
{
    "bleu": 0.22517720618985276,
    "classes": {
        "gram": {
            "f1": 0.3959357587676172,
            "prec": 0.3491329479768786,
            "rec": 0.45722937168811506
        },
        "stem": {
            "f1": 0.30368225072403804,
            "prec": 0.41704545454545455,
            "rec": 0.23877683799609628
        }
    },
    "morpheme_level": {
        "accuracy": 0.3397480755773268,
        "average_accuracy": 0.3601194508154002
    },
    "word_level": {
        "accuracy": 0.29954809554551326,
        "average_accuracy": 0.28599925776649926
    }
}


In [ ]:
def restore_punctuation(original, predicted):
    pred_line = []
    for orig, pred in zip(original.split('\t'), predicted.split('\t')):
        new_pred = pred + orig[len(pred.replace('-', '')):]
        pred_line.append(new_pred)
    return '\t'.join(pred_line)

In [ ]:
new_file = []
with open('23.txt', 'r') as file:
    for num, line in enumerate(file):
        line = line.strip()
        number_match_segm = re.match('\d+\>\t', line)
        other = re.match('\d+[\<=]\t', line)
        # new_line = re.sub('\[[А-ЯЁ]+\:\]\t', '', line)

        if number_match_segm:
            number = re.match('\d+', number_match_segm.group()).group()
            speech = re.search('\[[А-ЯЁ]+\:\]', line)

            new_line = re.sub(number_match_segm.group(), '', line)
            # new_pred_line = re.sub('[\.\,\?\!]+', '', new_line)
            new_line = re.sub('\[[А-ЯЁ]+\:\]\t', '', new_line)
            new_pred_line = re.sub('[\.\,\?\!\:]+', '', new_line)

            new_pred = glossator.gloss_sent(new_pred_line)
            new_segm = restore_punctuation(new_line, new_pred['segmentation'])
            if speech:
                speech = speech.group() + '\t'
            else:
                speech = ''
            new_file.append(f'{number}>\t{speech}{new_segm}')
            new_gloss = new_pred['glossing']
            new_file.append(f'{number}#\t{speech}{new_gloss}')
        else:
            new_file.append(line)

In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 11.7 MB/s eta 0:00:00


In [ ]:
import sacrebleu

In [ ]:
from sacrebleu.metrics import CHRF

In [ ]:
def compute_chrf(true_label, pred_label):
    """
    Вычисляет метрику chrF между true_label и pred_label.

    :param true_label: Истинная метка (str).
    :param pred_label: Предсказанная метка (str).
    :return: Значение chrF (float).
    """
    chrf = CHRF()
    score = chrf.corpus_score([pred_label], [[true_label]])
    return score.score

In [ ]:
final_metric = 0
for pred, gold in zip(gold_glossing, pred_glossing):

    final_metric += compute_chrf(gold, pred)

print(f'ChrF metric for segmentation: {final_metric/len(gold_glossing):.2f}')

ChrF metric for glossing: 52.63


In [ ]:
pred_glossing[0]

'быть.таким-CONV:ANT.3.PL\tUNK-EMPH.3.PL\tUNK\tесть-EMPH.3.PL\tбыть.таким-CONV:ANT.3.PL'

In [ ]:
final_metric = 0
for pred, gold in zip(gold_glossing, pred_glossing):
    true_pred = re.sub('[А-яё]+(\.[А-яё])*', 'STEM', pred)
    true_pred = re.sub('(UNK|ADJ|PRON|DET|NOUN|VERB|ADV)+', 'STEM', true_pred)
    true_gold = re.sub('(UNK|ADJ|PRON|DET|NOUN|VERB|ADV)+', 'STEM', gold)
    true_gold = re.sub('[А-яё]+(\.[А-яё])*', 'STEM', true_gold)
    final_metric += compute_chrf(true_gold, true_pred)

print(f'ChrF metric for glossing (only grammatical morphs): {final_metric/len(gold_glossing):.2f}')

ChrF metric for glossing (only grammatical morphs): 80.48
